In [173]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score

In [160]:
df = pd.read_csv(".\EEG_EMOTION\dataset_de_LDS_SEEDIV.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [161]:
df

0          1          2          3          4  \
id_user session video                                                          
10      1       1      25.622467  22.612988  21.225053  19.946429  19.266408   
                2      25.496412  22.407150  20.974761  20.547296  20.101432   
                3      25.302885  22.570660  21.633718  21.230311  21.011355   
                4      25.939116  22.808869  21.332588  19.427715  18.515616   
                5      25.503216  22.350826  21.157540  19.464702  18.714055   
...                          ...        ...        ...        ...        ...   
9       3       20     25.031688  22.242642  21.493354  20.546762  20.240579   
                21     25.676278  22.638414  21.002611  20.324261  20.132503   
                22     25.323145  22.412124  21.821257  20.533943  20.393856   
                23     24.879851  22.047250  21.202716  20.121354  19.557465   
                24     25.481525  22.404174  21.688045  20.494014  20.080786   

                               5          6          7          8          9  \
id_user session video                                                          
10      1       1      23.396372  20.964809  20.035117  17.815367  16.887869   
                2      23.023210  21.083666  19.698849  17.761071  16.874142   
                3      23.156935  21.018987  20.335182  18.201790  17.557549   
                4      22.954357  20.906463  20.060412  17.828035  16.748080   
                5      23.024172  20.880395  20.269262  17.854670  16.799909   
...                          ...        ...        ...        ...        ...   
9       3       20     21.793385  18.860982  18.035320  16.896775  16.196043   
                21     21.736976  18.712277  17.463331  16.739365  16.433142   
                22     21.565260  18.816661  18.326217  17.525892  17.510356   
                23     21.748668  18.707098  17.733498  16.605061  15.808999   
                24     21.794618  18.929078  18.257980  16.976043  16.482274   

                       ...  1590  1591  1592  1593  1594  1595  1596  1597  \
id_user session video  ...                                                   
10      1       1      ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                2      ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                3      ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                4      ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                5      ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
9       3       20     ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                21     ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                22     ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                23     ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
                24     ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                       1598  1599  
id_user session video              
10      1       1       NaN   NaN  
                2       NaN   NaN  
                3       NaN   NaN  
                4       NaN   NaN  
                5       NaN   NaN  
...                     ...   ...  
9       3       20      NaN   NaN  
                21      NaN   NaN  
                22      NaN   NaN  
                23      NaN   NaN  
                24      NaN   NaN  

[1080 rows x 1601 columns]

In [162]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [170]:
scores_test = []
scores_training = []
clfSD = HistGradientBoostingClassifier(loss="log_loss",learning_rate=0.1, max_depth=50, random_state=22)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [171]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.4861111111111111
TRAIN: 1.0


In [174]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

logo = LeaveOneGroupOut()

scores_test = []
gbc_SI = HistGradientBoostingClassifier(loss="log_loss",learning_rate=0.1, max_depth=50, random_state=22)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

print("TEST:", np.array(scores_test).mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

clf = HistGradientBoostingClassifier(loss="log_loss",learning_rate=0.1, max_depth=50, random_state=22).fit(X_trainSB, y_trainSB)

print("TEST", clf.score(X_testSB, y_testSB))
print("TRAIN", clf.score(X_trainSB, y_trainSB))